# ModOAP - Segmentation de texte en blocs au format json

Ce carnet propose de segmenter le texte de fichiers TXT ou PDF (océrisés) contenus dans un dossier source. 

Pour chaque fichier, le résultat est un fichier au format JSON créé dans le dossier de destination, comportant les blocs de texte segmentés. 

In [1]:
#@markdown ### Synchronisation d'un compte Google Drive :
#@markdown ##### Lancer cette cellule, puis cliquer sur le lien généré par Google et copier/coller le code si demandé

try :
  import fitz  # this is pymupdf
except ModuleNotFoundError :
  !pip -q install pymupdf
  import fitz

from google.colab import drive
import re
import os
import json
import glob



     |████████████████████████████████| 6.4MB 182kB/s 


In [ ]:

#@markdown ### Entrez le chemin du dossier contenant les fichiers txt ou pdf :
source_dir = "" #@param {type:"string"}

#@markdown Possibilité de copier/coller le chemin depuis la fenêtre de gauche : onglet "Fichiers" -> clic droit sur un dossier -> "Copier le chemin"

#@markdown Exemple de chemin:
#@markdown /content/drive/My Drive/fichiers/

#@markdown ---

#@markdown ### Entrez le chemin du dossier de destination où créer les fichiers json :
destination_dir = "" #@param {type:"string"}

#@markdown ---


files = [fichier for fichier in glob.glob(os.path.join(source_dir,"*.*"))]


if not os.path.exists(destination_dir):
  os.mkdir(destination_dir)


for fichier in files :
  titre = fichier.split("/")[-1]
  titre = titre.split(".")[0]
  text_blocks = {}

  if fichier.endswith(".pdf") :
    contenus_pages = []
    with fitz.open(fichier) as doc:
      for page in doc :
        contenus_pages.append(page.getText())
    total_pages = len(contenus_pages)
    for num_page in range(total_pages) :
      i = 0
      contenus_pages[num_page] = re.sub("\n", r" ", contenus_pages[num_page]) 
      contenu_page = re.split('!|\.+|\?',contenus_pages[num_page])
      #print(contenu_page)
      for tb in contenu_page :
        if len(tb) > 2 :
          id = "PAG_"+str(num_page)+"_TB"+str(i)
          dico_id = {"Page_Num" : num_page, "Content" : tb}
          text_blocks[id] = dico_id
          i += 1
    info_doc = {"Titre" : titre, "Total_Pages" : total_pages }
    jsondic = {"Infos_Doc" : info_doc, "Text_Blocks" : text_blocks} 
    with open(os.path.join(destination_dir,titre+"_texte_structure.json"), 'w') as jout :
      json.dump(jsondic, jout)
    print("Le texte du document a été sauvegardé dans le fichier ", os.path.join(destination_dir,titre+"_texte_structure.json"))


  elif fichier.endswith(".txt") :
    with open(fichier, "r") as file_in :
      contenu = file_in.read()
    
    contenu_tbs = re.sub("\n", r" ", contenu)
    
    contenu_tbs = re.split('!|\.+|\?',contenu_tbs)
    i = 0
    for tb in contenu_tbs :
      
      if len(tb) > 2 :
        id = "TB"+str(i)
        dico_id = {"Content" : tb}
        text_blocks[id] = dico_id
        i += 1
    info_doc = {"Titre" : titre }
    jsondic = {"Infos_Doc" : info_doc, "Text_Blocks" : text_blocks} 
    with open(os.path.join(destination_dir,titre+"_texte_structure.json"), 'w') as jout :
      json.dump(jsondic, jout)
    print("Le texte du document a été sauvegardé dans le fichier ", os.path.join(destination_dir,titre+"_texte_structure.json"))

  else : print(fichier, " : le type de fichier n'est pas supporté ")
